In [55]:
!nvidia-smi

Sun Jul  4 05:44:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
path='/content/drive/MyDrive/ColabNotebooks/native_language_classification_bert/'
!cd $path && pwd && rm bert_input_data/*.csv

In [ ]:
  ORIGIN_DATA_DIR='/content/drive/MyDrive/ColabNotebooks/native_language_classification_bert/bert_input_data'
  BERT_FEATURE_DIR='/content/drive/MyDrive/ColabNotebooks/native_language_classification_bert/bert_output_data/'
  BERT_DATA_DIR='/content/drive/MyDrive/ColabNotebooks/native_language_classification_bert/BERT_DATA_DIR/'

In [ ]:
path='/content/drive/MyDrive/ColabNotebooks/native_language_classification_bert/'
import pandas as pd
import os
os.chdir(path)
os.getcwd()

In [ ]:
train_df = pd.read_csv('original_data/lang_id_train.csv')
train_df = train_df.drop(['native_language'],axis=1)
train_df.to_csv('bert_input_data/lang_id_train.csv',header=0,index="FALSE")

In [16]:
for i in ["lang_id_eval","lang_id_test"]:
  df = None
  df = pd.read_csv('original_data/{}.csv'.format(i))
  df = df.drop(['native_language'],axis=1)
  df.to_csv('bert_input_data/{}.csv'.format(i), header=0,index="FALSE")

In [ ]:
train_df = pd.read_csv('original_data/lang_id_train.csv')
train_df = train_df.drop(['native_language'],axis=1)
train_df.to_csv('bert_input_data/lang_id_train.csv',header=0,index="FALSE")

In [36]:
train_df = pd.read_csv('bert_input_data/lang_id_train.csv')
train = pd.read_csv('original_data/lang_id_train.csv')

In [65]:
eval = pd.read_csv('original_data/lang_id_eval.csv')

In [67]:
eval.head()

,native_language,text
0,Japanese,What you can do for your health and I like it ...
1,Japanese,How to use `peanut butter ' Hello ! I have a q...
2,Japanese,earthquake I went to listen to a lecture meeti...
3,Japanese,The Questions There are many abbrebiations and...
4,Japanese,"Long time no see!!! I""m very tied! Many part t..."


In [66]:
test = pd.read_csv('original_data/lang_id_test.csv')

In [68]:
test.head()

,native_language,text
0,Japanese,"got an official job offer Today, I decided to ..."
1,Japanese,I am bum. I am bum.
2,Japanese,exam... I will have to take the exam tomorrow....
3,Japanese,Self-introduction and my situation I work in f...
4,Japanese,Tough Schedule Hi there. So tough schedule las...


In [37]:
train.head()

,native_language,text
0,Japanese,"who, whom, whose I have often thinking that, I..."
1,Japanese,books I love reading. It's been one of my hobb...
2,Japanese,change job I want to change job. The company t...
3,Japanese,the origin of Japanese I like to learn about a...
4,Japanese,"Marunouchi Today, I am going go to Marunouchi,..."


In [38]:
train_df.head()

,0,"who, whom, whose I have often thinking that, I sometimes don't come to understand how to use ""who"", ""whose"", ""whom"". What does the difference these words?"
0,1,books I love reading. It's been one of my hobb...
1,2,change job I want to change job. The company t...
2,3,the origin of Japanese I like to learn about a...
3,4,"Marunouchi Today, I am going go to Marunouchi,..."
4,5,My pleasure My pleasure. I wrote diaries twice...


In [ ]:
!git clone https://github.com/google-research/bert.git

In [ ]:
!pwd

In [ ]:
!pip list installed|grep -i tensorflow
pkgs=!pip list installed|grep -i tensorflow|awk '{print $1}'
!echo $pkgs

In [ ]:
for p in pkgs:
  !pip uninstall $p --yes

In [ ]:
pip install tensorflow-gpu==1.14.0

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
!python3 bert/extract_features.py --help

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

In [ ]:
!unzip uncased_L-12_H-768_A-12.zip

In [ ]:
!python bert/extract_features.py --vocab_file uncased_L-12_H-768_A-12/vocab.txt --bert_config_file uncased_L-12_H-768_A-12/bert_config.json --init_checkpoint uncased_L-12_H-768_A-12/bert_model.ckpt --max_seq_length 128 --batch_size 8 --layers -1 --input_file bert_input_data/lang_id_train.csv --output_file bert_output_data/lang_id_train.jsonlines

In [ ]:
!for i in lang_id_eval lang_id_test; do python bert/extract_features.py --vocab_file uncased_L-12_H-768_A-12/vocab.txt --bert_config_file uncased_L-12_H-768_A-12/bert_config.json --init_checkpoint uncased_L-12_H-768_A-12/bert_model.ckpt --max_seq_length 128 --batch_size 8 --layers -1 --input_file bert_input_data/$i.csv --output_file bert_output_data/$i.jsonlines;done

In [ ]:
!cat /etc/lsb-release
!apt-get install jq

In [ ]:
!head -1 bert_output_data/lang_id_train.jsonlines|jq . 

In [58]:
def get_vectors(file:str):
  bert_vectors=[]
  with open(file,'rt') as infile:
    for line in infile:
      bertdata = json.loads(line)
      for t in bertdata['features']:
        # only [CLS] token embedding for classification taks
        if t['token'] == '[CLS]':
          # only final layers representation
          vec = t['layers'][0]['values']
          bert_vectors.append(vec)
          break
  
  return bert_vectors

In [ ]:
bert_vectors_train = get_vectors('bert_output_data/lang_id_train.jsonlines')

In [59]:
import json
bert_vectors_eval = get_vectors('bert_output_data/lang_id_eval.jsonlines')
bert_vectors_test = get_vectors('bert_output_data/lang_id_test.jsonlines')

In [35]:
print(len(bert_vectors))
len(bert_vectors[0])

6000


768

In [60]:
print(len(bert_vectors_eval))
len(bert_vectors_eval[0])

2000


768

In [61]:
print(len(bert_vectors_test))
len(bert_vectors_test[0])

2000


768

In [40]:
import numpy as np
x_train=np.array(bert_vectors)
y_train=train['native_language'].values

In [62]:
x_test=np.array(bert_vectors_test)

In [52]:
x_train.shape

(6000, 768)

In [50]:
y_train.shape

(6000,)

In [51]:
y_train

array(['Japanese', 'Japanese', 'Japanese', ..., 'Arabic', 'Arabic',
       'Arabic'], dtype=object)

In [56]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(penalty='l2',C=1.0, max_iter=2000)
model_fit = lr_model.fit(x_train,y_train)

In [63]:
predict_y=lr_model.predict(x_test)

In [64]:
predict_y

array(['Japanese', 'Korean', 'Japanese', ..., 'Mandarin', 'Mandarin',
       'Russian'], dtype=object)

In [69]:
y_test=test['native_language'].values
y_test

array(['Japanese', 'Japanese', 'Japanese', ..., 'Arabic', 'Arabic',
       'Arabic'], dtype=object)

In [73]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predic_y)

0.3535

In [75]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predict_y))

              precision    recall  f1-score   support

      Arabic       0.37      0.13      0.19       200
   Cantonese       0.29      0.17      0.22       200
    Japanese       0.45      0.81      0.58       200
      Korean       0.24      0.34      0.28       200
    Mandarin       0.10      0.12      0.11       200
      Polish       0.53      0.31      0.39       200
     Russian       0.57      0.56      0.56       200
     Spanish       0.53      0.34      0.42       200
        Thai       0.68      0.56      0.62       200
  Vietnamese       0.12      0.20      0.15       200

    accuracy                           0.35      2000
   macro avg       0.39      0.35      0.35      2000
weighted avg       0.39      0.35      0.35      2000



In [76]:
from sklearn import metrics
cm = metrics.confusion_matrix(y_test,predict_y)
print(cm)

[[ 26   1   2   4 126   6   6   4   3  22]
 [  3  34   3  14  31   2   6   2   9  96]
 [  3   4 162  12   6   1   3   1   1   7]
 [  1   6  90  68   9   4   5   5   8   4]
 [ 11  41  27  58  24   7  11   6   6   9]
 [  4   1  12   6  17  62  27   9   3  59]
 [  2   3   7  39   3  14 111  16   1   4]
 [  8   1  10   7   9   7  13  69   4  72]
 [  5   8   9  33   6   2   2   4 112  19]
 [  8  17  35  39   8  12  10  15  17  39]]
